In [1]:
# @title Setup

competition = "extractive-qa"  # @param
# @markdown ---

from google.colab import userdata
import json

# Get the Kaggle credentials from Colab's userdata
username = userdata.get("KAGGLE_USER")
key = userdata.get("KAGGLE_KEY")

# Echo the credentials into the kaggle.json file
!mkdir -p ~/.kaggle
!echo '{{"username":"{username}","key":"{key}"}}' > ~/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

competition_name = "individual-test-extraction-question-answering"
dir_name = "extractive-qa"

! kaggle competitions download -c {competition_name}
! mkdir {dir_name}
! unzip /content/{competition_name}.zip -d {dir_name}

  0% 0.00/168k [00:00<?, ?B/s]
100% 168k/168k [00:00<00:00, 500MB/s]
Archive:  /content/individual-test-extraction-question-answering.zip
  inflating: extractive-qa/context/1345136.txt  
  inflating: extractive-qa/context/1345137.txt  
  inflating: extractive-qa/context/1345138.txt  
  inflating: extractive-qa/context/1345139.txt  
  inflating: extractive-qa/context/1345140.txt  
  inflating: extractive-qa/context/1345141.txt  
  inflating: extractive-qa/context/1345142.txt  
  inflating: extractive-qa/context/1345143.txt  
  inflating: extractive-qa/context/1345144.txt  
  inflating: extractive-qa/context/1345145.txt  
  inflating: extractive-qa/context/1345146.txt  
  inflating: extractive-qa/context/1345147.txt  
  inflating: extractive-qa/context/1345148.txt  
  inflating: extractive-qa/context/1345149.txt  
  inflating: extractive-qa/context/1345150.txt  
  inflating: extractive-qa/context/1345153.txt  
  inflating: extractive-qa/context/1345154.txt  
  inflating: extractive-qa/co

In [2]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [9]:
!pip install -q sentence-transformers FlagEmbedding

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.9/163.9 kB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 866.1/866.1 kB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.0/135.0 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.5 MB/s eta 0:00:00


In [1]:
from unsloth import FastLanguageModel
from sentence_transformers import SentenceTransformer, util
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM, TextStreamer
import torch

import pandas as pd
import numpy as np

from tqdm import tqdm
import glob
import os

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


## Data Prep

In [2]:
documents = []
doc_files = np.sort(glob.glob('/content/extractive-qa/context/*.txt'))
for file in doc_files:
    with open(file, 'r') as f:
        text = ""
        for line in f:
            text += line

        documents.append(text)


# convert to dataframe
doc_df = pd.DataFrame({'id': doc_files, 'content': documents})
doc_df['id'] = doc_df['id'].apply(lambda x: os.path.basename(x))

print(doc_df.shape)
doc_df.head()

(58, 2)


,id,content
0,1345136.txt,การซื้อขายตราสารหนี้ในไทย\n\n1.\tตลาดตราสารหนี...
1,1345137.txt,การซื้อขายตราสารหนี้ในไทย\n\n2.\tกระบวกการออกต...
2,1345138.txt,การซื้อขายตราสารหนี้ในไทย\n\n3.\tผู้เกี่ยวข้อง...
3,1345139.txt,การซื้อขายตราสารหนี้ในไทย\n\n4.\tดัชนีตราสารหน...
4,1345140.txt,การซื้อขายตราสารหนี้ในไทย\n\n5.\tข้อมูลและแหล่...


In [3]:
from FlagEmbedding import BGEM3FlagModel

emb_model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)
doc_df['embeddings'] = doc_df['content'].apply(lambda x: emb_model.encode(x, batch_size=32)['dense_vecs'])

train_df = pd.read_csv('/content/extractive-qa/train.csv')

print(train_df.shape)
train_df.head()

(410, 5)


,file,question,answer,answer_start,answer_end
0,1345136.txt,ใครเป็นผู้ออกตราสารหนี้ภาคเอกชน ไร้ใบตราสาร,ไม่มีการรวมศูนย์อย่างตราสารภาครัฐ แต่ส่วนใหญ่ด...,7716,7799
1,1345195.txt,Penny Stock หุ้นเหรียญ มีลักษณะอย่างไร,*\tหุ้นที่มีราคาซื้อขายต่อหน่วยต่ำ (ในสหรัฐฯ ค...,3019,3138
2,1345200.txt,ราคาต้นงวดคืออะไร,ราคาต้นทุนหรือราคาซื้อ,501,522
3,1345179.txt,อัตราการใช้สิทธิของ NVDR คือเท่าไหร่,1 : 1 (NVRD : สินทรัพย์อ้างอิง) เสมอ,1012,1047
4,1345158.txt,หนังสือชี้ชวนส่วนสรุปข้อมูลสำคัญ (Fund Fact Sh...,เอกสารที่ให้ข้อมูลที่สำคัญกับผู้ลงทุน,1116,1152


In [4]:
def manual_indexing(sentence):
    compare_df = doc_df.copy()

    sent_embedding = emb_model.encode(sentence, batch_size=32)['dense_vecs']
    compare_df['cos_sim'] = doc_df['embeddings'].apply(lambda x: sent_embedding@x.T)

    return compare_df[ compare_df['cos_sim'] == compare_df['cos_sim'].max() ]['content'].values[0]

manual_indexing(train_df.loc[0, 'question'])

'การซื้อขายตราสารหนี้ในไทย\n\n2.\tกระบวกการออกตราสารหนี้ (ในตลาดแรก)\n2.1\tการออกตราสารหนี้ภาครัฐ (ตั๋วเงินคลัง, พันธบัตรรัฐบาล, พันธบัตรธนาคารแห่งประเทศไทย)\n\t1.\tกระบวนการออก\n\t\t*\tธนาคารแห่งประเทศไทยเป็นผู้จัดการประมูล\n\t\t*\tธนาคารแห่งประเทศไทยจะประกาศตารางการประมูลทุกปลายเดือนในเว็บไซต์\n\t\t*\tสมาคมตราสารหนี้ เป็นผู้กำหนดสัญลักษณ์ของตราสาร\n\t2.\tการจัดจำหน่าย\n\t\t1)\tการประมูล มี 2 วิธี ดังนี้\n\t\t\t*\tการประมูลแบบแข่งขันราคา (Competitive biding) เปิดให้กับผู้มีสิทธิเข้าประมูล เสนออัตราผลตอบแทน (Yield) ของตราสาร โดยผู้ที่เสนออัตราผลตอบแทนต่ำสุดจะได้รับการจัดสรรตราสารก่อน (การที่ผู้ลงทุนได้อัตราผลตอบแทนต่ำสุด อีกมุมมองหนึ่งหมายถึงการที่ผู้ออกตราสารจ่ายดอกเบี้ยต่ำสุดนั่นเอง) จากนั้นตราสารที่เหลือจะถูกจัดสรรให้กับผู้ที่เสนออัตราผลตอบแทนต่ำสุดลำดับถัดไป จะเห็นได้ว่าการประมูลลักษณะนี้ผู้ลงทุนแต่ละรายจะได้อัตราผลตอบแทนจาก ตราสารไม่เท่ากัน ซึ่งการประมูลลักษณะนี้เรียกว่า (Multiple Price/American Auction)\n\t\t\t*\tการประมูลแบบไม่แข่งขันราคา (Non-competitive biding) วิธีการนี้ผู้ปร

## Test set

In [5]:
test_df = pd.read_csv('/content/extractive-qa/test.csv')

print(test_df.shape)
test_df.head()

(275, 2)


,id,question
0,1,ตลาดหลักทรัพย์ได้กำหนดราคาสูงสุดต่ำสุดของ Warr...
1,2,ลักษณะแนวทางในการระดมทุนมีอะไรบ้าง
2,3,Gross Price หรือ Dirty Price มีวิธีการจ่ายดอกเ...
3,4,รายได้จากการลงทุนต่อ (Reinvestment income) คือ...
4,5,ตัวอย่างลำดับเหตุการณ์ที่เกี่ยวข้องกับการจ่ายเ...


In [6]:
prompt = """คุณเป็นนักธุรกิจผู้มีความเชี่ยวชาญในด้านการลงทุน คุณมีหน้าที่ในการตอบคำตอบการลงมุนแบบ extractive
จาก documents ที่แนบไปให้ด้วยกับคำถาม โดยดึงส่วนที่เป็นคำตอบจากใน documents มาตอบ
**หากไม่รู้ ไม่มั่นใจหรือคิดว่า documents ที่แนบไปด้วยไม่มีความเกี่ยวข้องกับคำถาม ให้ตอบว่า NaN**

## Question
{}

## Documents
{}

/no_think
"""

test_set = []
for q in test_df['question']:
    message = [{
        "role": "user", "content": prompt.format(q, manual_indexing(q))
    }]
    test_set.append(message)

test_set[0]

[{'role': 'user',
  'content': 'คุณเป็นนักธุรกิจผู้มีความเชี่ยวชาญในด้านการลงทุน คุณมีหน้าที่ในการตอบคำตอบการลงมุนแบบ extractive\nจาก documents ที่แนบไปให้ด้วยกับคำถาม โดยดึงส่วนที่เป็นคำตอบจากใน documents มาตอบ\n**หากไม่รู้ ไม่มั่นใจหรือคิดว่า documents ที่แนบไปด้วยไม่มีความเกี่ยวข้องกับคำถาม ให้ตอบว่า NaN**\n\n## Question\nตลาดหลักทรัพย์ได้กำหนดราคาสูงสุดต่ำสุดของ Warrant อย่างไร\n\n## Documents\nตราสารที่เชื่อมโยงกับตราสารทุน\n\n3.\tใบสำคัญแสดงสิทธิ (Warrant)\n3.1\tความหมายและเงื่อนไขต่างๆ ของ Warrant (วอแรนต์)\n\t1.\tความหมายโดยสากล Warrant เป็น Option (ออปชั่น) ประเภทหนึ่ง ที่มีลักษณะเป็นสัญญาที่ให้สิทธิในการซื้อหรือขายสินทรัพย์อ้างอิง (Underlying Asset) ในระยะเวลาที่กำหนด เช่นเดียวกับ DW โดย Warrant ส่วนใหญ่จะอ้างอิงกับหุ้นสามัญ\n\t2.\tความหมายในไทย Warrant เป็นสัญญาที่ให้สิทธิในการจะซื้อหุ้นสามัญอ้างอิง ในระยะเวลาที่กำหนด (ระยะเวลาส่วนใหญี่อยู่ระหว่าง 2 – 10 ปี) โดยผู้ออก Warrant จะเป็นบริษัทที่จดทะเบียนในตลาดหลักทรัพย์นั่นเอง โดยเนื้อหาต่อจากนี้จะกล่าวถึง Warrant ไทยเท่านั้น\n\

In [7]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-8B-unsloth-bnb-4bit",
    max_seq_length = 2048,
    dtype = torch.bfloat16,
    load_in_4bit = False,
    load_in_8bit = False,
)

==((====))==  Unsloth 2025.6.12: Fast Qwen3 patching. Transformers: 4.53.0.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [ ]:
FastLanguageModel.for_inference(model)

answer = []
for prompt in tqdm(test_set):
    prompt = tokenizer.apply_chat_template(
        prompt,
        enable_thinking=False,
        tokenize=False
    )

    # tokenize and feed into model
    inputs = tokenizer(prompt, return_tensors = "pt").to("cuda")

    text_streamer = TextStreamer(tokenizer, skip_prompt=True)
    outputs = model.generate(**inputs,
                            #  streamer=text_streamer,
                             max_new_tokens=256, temperature=0.2, top_p=0.6, top_k=16)
    decoded = tokenizer.batch_decode(outputs)
    answer.append(decoded)

 91%|█████████ | 249/275 [1:00:06<09:28, 21.87s/it]

In [ ]:
pred = [a[0].split('\n\n')[-1].replace('<|im_end|>', '') for a in answer]
pred[:10]

['NaN',
 'NaN',
 'คำตอบจากเอกสาร: **Gross Price หรือ Dirty Price มีวิธีการจ่ายดอกเบี้ยโดยรวมดอกเบี้ยค้างรับ (Accrued Interest) ที่ผู้ขายควรได้รับตามระยะเวลา',
 'รายได้จากการลงทุนต่อ (Reinvestment income): รายได้จากการนำดอกเบี้ยรับไปลงทุนต่อ',
 'NaN',
 'NaN',
 'NaN',
 'NaN',
 'NaN',
 'NaN']

In [ ]:
def postprocess_data(sentence):
    if 'NaN' in sentence or sentence.count('ม') == 256 or sentence.count('-in') == 256:
        return np.nan
    else:
        return sentence.replace('/no_think\n\n', '')

final_answer = [postprocess_data(p) for p in pred]
print(final_answer[:5])

[nan, nan, 'คำตอบจากเอกสาร: **Gross Price หรือ Dirty Price มีวิธีการจ่ายดอกเบี้ยโดยรวมดอกเบี้ยค้างรับ (Accrued Interest) ที่ผู้ขายควรได้รับตามระยะเวลา', 'รายได้จากการลงทุนต่อ (Reinvestment income): รายได้จากการนำดอกเบี้ยรับไปลงทุนต่อ', nan]


## Final Submission

In [ ]:
submission_df = pd.read_csv('/content/extractive-qa/sample_submission.csv')
submission_df.loc[3:, 'answer'] = final_answer[3:]
submission_df

,id,answer
0,1,สูงสุด +30% ต่ำสุด -30% ของราคาปิดสินทรัพย์อ้า...
1,2,NaN
2,3,"จะจ่ายเป็น รายเดือน, รายไตรมาส, ทุกครึ่งปี หรื..."
3,4,รายได้จากการลงทุนต่อ (Reinvestment income): รา...
4,5,NaN
...,...,...
270,271,8.\tการจัดการกับเหตุการณ์ที่เกี่ยวข้องกับระบบก...
271,272,NaN
272,273,13.\tการประเมินมูลค่าของหุ้น\nการประเมินมูลค่า...
273,274,NaN


In [ ]:
submission_df.isnull().sum()

,0
id,0
answer,206


In [ ]:
submission_df.to_csv('Naive-RAG_Qwen3-8B.csv', index=False)